In [1]:
#导入必要的库即版本

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras
#import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd,sklearn,tf,keras:
    print(module.__name__, module.__version__)

2.0.0-beta1
sys.version_info(major=3, minor=7, micro=0, releaselevel='final', serial=0)
matplotlib 2.2.3
numpy 1.16.4
pandas 0.23.4
sklearn 0.19.2
tensorflow 2.0.0-beta1
tensorflow.python.keras.api._v2.keras 2.2.4-tf


In [ ]:
calss_names = []
train_labels_file = 

In [ ]:
train_dir = './cifar10/train/'
valid_dir = '../cifar10/eval/'
label_file = '../cifar10/trainLabels.csv'
print(os.path.exists(train_dir))

print(os.listdir(train_dir))
print(os.listdir(valid_dir))

In [ ]:
# 读取数据
labels = pd.read_csv(label_file, header=0)
print(labels)

In [ ]:
# 读取图片
height = 128
width =128   # 读取图片后对图片进行缩放,固定大小
channels = 3
batch_size = 64
num_classes = 10
# keras generator读取数据
train_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255.,  # 像素转为0-1
    rotation_range=40,# 图片随机旋转角度，40代表-40——40之间
    width_shift_range=0.2,# 水平位移，训练模型时使用数据和位移之后的数据
    height_shift_range=0.2,# 垂直位移，（0，1）之间的数代表比例，大于1代表像素值，在0-0.2之间随机选择一个数进行位移
    shear_range=0.2, # 剪切强度
    zoom_range=0.2, # 缩放强度
    horizontal_flip=True,# 随机进行水平翻转
    fill_mode='nearest',# 填充像素规则，如放大操作，nearest代表最近的真实像素
    
)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                  target_size=(height,width),
                                                  batch_size=batch_size,
                                                  seed=7,
                                                  shuffle=True,
                                                   #设置label格式，one-hot编码前，one-hot编码后
                                                  class_mode='categorical')

valid_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255.)
valid_generator = valid_datagen.flow_from_directory(valid_dir,
                                                   target_size=(height,width),
                                                   batch_size=batch_size,
                                                   seed=7,
                                                   shuffle=True,
                                                   class_mode='categorical'
)
# 训练集、验证集数据量
train_num = train_generator.samples
valid_num = valid_generator.samples
print(train_num, valid_num)

In [ ]:
for i in range(2):
    x,y = train_generator.next()
    print(x.shape, y.shape)
    print(y)

In [ ]:
# 基础模型搭建与训练
model = keras.models.Sequential([
    keras.layers.Conv2D(filters=32,kernel_size=3,padding='same',
                       activation='relu',input_shape=[width,height,channels]),
    keras.layers.Conv2D(filters=32,kernel_size=3,padding='same',
                       activation='relu'),
    keras.layers.MaxPool2D(pool_size=2),
    
    keras.layers.Conv2D(filters=64,kernel_size=3,padding='same',
                       activation='relu'),
    keras.layers.Conv2D(filters=64,kernel_size=3,padding='same',
                       activation='relu'),
    keras.layers.MaxPool2D(pool_size=2),
    
    keras.layers.Conv2D(filters=128,kernel_size=3,padding='same',
                       activation='relu'),
    keras.layers.Conv2D(filters=128,kernel_size=3,padding='same',
                       activation='relu'),
    keras.layers.MaxPool2D(pool_size=2),
    
    keras.layers.Flatten(),
    keras.layers.Dense(128,activation='relu'),
    keras.layers.Dense(num_classes,activation='softmax')
])

model.compile(loss='categorical_crossentropy',
             optimizer='adam',metrics=['accuracy'])

model.summary()

In [ ]:
epochs = 10
# 数据由generator产生
history = model.fit_generator(train_generator,
                             steps_per_epoch=train_num//batch_size,
                             epochs=epochs,
                             validation_data=valid_generator,
                             validation_steps=valid_num//batch_size)



In [ ]:
print(history.history.keys())

In [ ]:
def plot_learning_curves(history, label, epochs, min_value, max_value):
    data={}
    data[label] = history.history[label]
    data['val_'+label] = history.history['val_'+label]
    pd.DataFrame(data).plot(figsize=(8,5))
    plt.grid(True)
    plt.axis([0,epochs,min_value,max_value])
    plt.show()
    
plot_learning_curves(history,'acc',epochs,0,1)
plot_learning_curves(history,'loss',epochs,1.5,2.5)